In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    get_load_config_from_yaml,
)
from importlib import reload
import core
import keras
import core.keras_models as keras_models
import core.utils as utils

MODEL_DIR = f"models/assignment_transformer/"
CONFIG_PATH = "../config/nominal_load_config.yaml"

import os

if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


data_config = DataProcessor.load_from_npz(
    load_config.data_path, max_events=6_000_000, event_numbers="even"
)

X, y = DataProcessor.get_data()
del DataProcessor  # Free memory

In [ ]:
FeatureConcatAssigner = keras_models.FeatureConcatAssigner(data_config, name="FeatureConcatAssigner")
TransformerAssigner = keras_models.TransformerAssigner(data_config, name="TransformerAssigner")

In [ ]:
FeatureConcatAssigner.build_model(
    hidden_dim=128,
    num_layers=6,
    dropout_rate=0.1,
    compute_HLF=True,
    use_global_event_inputs=True,
    log_variables=True
)
FeatureConcatAssigner.adapt_normalization_layers(X)
FeatureConcatAssigner.compile_model(
    loss={
        "assignment": utils.AssignmentLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [utils.AssignmentAccuracy(name="assignment_accuracy")],
    },
    #loss_weights={"assignment": 1.0, "confidence_loss_output": 1.0},
)

In [ ]:
TransformerAssigner.build_model(
    hidden_dim=128,
    num_layers=6,
    dropout_rate=0.1,
    compute_HLF=True,
    use_global_event_inputs=True,
    log_variables=True
)
TransformerAssigner.adapt_normalization_layers(X)
TransformerAssigner.compile_model(
    loss={
        "assignment": utils.AssignmentLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [utils.AssignmentAccuracy(name="assignment_accuracy")],
    },
    #loss_weights={"assignment": 1.0, "confidence_loss_output": 1.0},
)

In [ ]:
X_train, y_train, sample_weights= FeatureConcatAssigner.prepare_training_data(X, y, sample_weights=utils.compute_sample_weights(X, y), copy_data=True)

In [ ]:
X_train, y_train, sample_weights= TransformerAssigner.prepare_training_data(X, y, sample_weights=utils.compute_sample_weights(X, y), copy_data=True)
TransformerAssigner.train_model(
    X_train,
    y_train,
    epochs=1,
    sample_weight=sample_weights,
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
    ],
    validation_split=0.1,
)

In [ ]:
FeatureConcatAssigner.train_model(
    X_train,
    y_train,
    epochs=1,
    sample_weight=sample_weights,
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
    ],
    validation_split=0.1,
)

In [ ]:
transformer_assigner_dir = os.path.join(MODEL_DIR, "transformer_assigner")
os.makedirs(transformer_assigner_dir, exist_ok=True)
TransformerAssigner.save_model(os.path.join(transformer_assigner_dir, "odd_model.keras"))

In [ ]:
feature_concat_dir = os.path.join(MODEL_DIR, "feature_concat_assigner")
os.makedirs(feature_concat_dir, exist_ok=True)

FeatureConcatAssigner.save_model(os.path.join(feature_concat_dir, "odd_model.keras"))